In [1]:
import torch
# assert '.'.join(torch.__version__.split('.')[:2]) == '1.4'
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F  # useful stateless functions
import numpy as np

In [2]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [3]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

In [4]:
class ResUNet_trial(nn.Module):
    def __init__(self, in_channel, num_classes):
        super().__init__()
        ########################################################################
        # TODO: Set up the layers you need for a three-layer ConvNet with the  #
        # architecture defined above.                                          #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.downsample1 = nn.Conv2d(3, 64, 3, stride=1, padding=1,bias=True)
        self.downsample2 = nn.Conv2d(64, 128, 3, stride=1, padding=1,bias=True)
        self.downsample3 = nn.Conv2d(128, 256, 3, stride=1, padding=1,bias=True)
        self.downsample4 = nn.Conv2d(256, 512, 3, stride=1, padding=1,bias=True)
        self.downsample5 = nn.Conv2d(512, 1024, 3, stride=1, padding=1,bias=True)
        
        self.int_conv_w1 = nn.Conv2d(1024,512,3,padding=1,bias=True)
        self.int_conv_w1 = nn.Conv2d(512,256,3,padding=1,bias=True)
        self.int_conv_w1 = nn.Conv2d(256,128,3,padding=1,bias=True)
        self.int_conv_w1 = nn.Conv2d(128,64,3,padding=1,bias=True)

        self.upsample1 = nn.ConvTranspose2d(1024, 512, 3, stride=1, padding=1, bias=True)
        self.upsample2 = nn.ConvTranspose2d(512, 256, 3, stride=1, padding=1, bias=True)
        self.upsample3 = nn.ConvTranspose2d(256, 128, 3, stride=1, padding=1, bias=True)        
        self.upsample4 = nn.ConvTranspose2d(128, 64, 3, stride=1, padding=1, bias=True)
        
        self.conv_w1 = nn.Conv2d(in_channels=in_channel,out_channels=64,kernel_size=3,padding=1,bias=True)
        nn.init.kaiming_normal_(self.conv_w1.weight)
        
        self.conv_w2 = nn.Conv2d(in_channels=64 ,out_channels=128,kernel_size=3,padding=1,bias=True)
        nn.init.kaiming_normal_(self.conv_w2.weight)

        self.conv_w3 = nn.Conv2d(in_channels=128 ,out_channels=256,kernel_size=3,padding=1,bias=True)
        nn.init.kaiming_normal_(self.conv_w3.weight)

        self.conv_w4 = nn.Conv2d(in_channels=256 ,out_channels=512,kernel_size=3,padding=1,bias=True)
        nn.init.kaiming_normal_(self.conv_w4.weight)

        self.conv_w5 = nn.Conv2d(in_channels=512 ,out_channels=1024,kernel_size=3,padding=1,bias=True)
        nn.init.kaiming_normal_(self.conv_w5.weight)

        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.deconv_w1 = nn.ConvTranspose2d(1024,512,kernel_size=3,stride=2,padding=1,bias=True)
        nn.init.kaiming_normal_(self.deconv_w1.weight)
        
        self.rconv_w1 = nn.Conv2d(1024 ,512,kernel_size=3,padding=1,bias=True)
        nn.init.kaiming_normal_(self.rconv_w1.weight)
        
        self.deconv_w2 = nn.ConvTranspose2d(512,256,kernel_size=3,stride=2,padding=1,bias=True)
        nn.init.kaiming_normal_(self.deconv_w2.weight)

        self.rconv_w2 = nn.Conv2d(512 ,256,kernel_size=3,padding=1,bias=True)
        nn.init.kaiming_normal_(self.rconv_w2.weight)
        
        self.deconv_w3 = nn.ConvTranspose2d(256,128,kernel_size=3,stride=2,padding=1,bias=True)
        nn.init.kaiming_normal_(self.deconv_w3.weight)
        
        self.rconv_w3 = nn.Conv2d(256 ,128,kernel_size=3,padding=1,bias=True)
        nn.init.kaiming_normal_(self.rconv_w3.weight)
        
        self.deconv_w4 = nn.ConvTranspose2d(128,64,kernel_size=3,stride=2,padding=1,bias=True)
        nn.init.kaiming_normal_(self.deconv_w4.weight)
        
        self.rconv_w4 = nn.Conv2d(128 , 64,kernel_size=3,padding=1,bias=True)
        nn.init.kaiming_normal_(self.rconv_w4.weight)

        self.conv_last = nn.Conv2d(64, num_classes, 1)


        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                          END OF YOUR CODE                            #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implement the forward function for a 3-layer ConvNet. you      #
        # should use the layers you defined in __init__ and specify the        #
        # connectivity of those layers in forward()                            #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****       
        
        
        conv_out_1 = F.relu(self.downsample1(x))
        print("Input shape : {}, Conv. 1 output shape : {} ".format(x.shape,conv_out_1.shape))

        max_pool_1 = F.max_pool2d(conv_out_1, 2)
        print("Max. pool 1 output shape : {}".format(max_pool_1.shape)) # [5, 64, 112, 112]

        conv_out_2 = F.relu(self.downsample2(max_pool_1))
        print("Conv. 2 output shape : {}".format(conv_out_2.shape))

        max_pool_2 = F.max_pool2d(conv_out_2, 2)
        print("Max pool 2 output shape : {}".format(max_pool_2.shape)) # [5, 128, 56, 56]

        conv_out_3 = F.relu(self.downsample3(max_pool_2))
        print("Conv. 3 output shape : {}".format(conv_out_3.shape))

        max_pool_3 = F.max_pool2d(conv_out_3, 2)
        print("Max pool 3 output shape : {}".format(max_pool_3.shape)) # [5, 256, 28, 28]

        conv_out_4 = F.relu(self.downsample4(max_pool_3))
        print("Conv. 4 output shape : {}".format(conv_out_4.shape))

        max_pool_4 = F.max_pool2d(conv_out_4, 2)
        print("Max pool 4 output shape : {}".format(max_pool_4.shape)) # [5, 512, 14, 14]

        conv_out_5 = F.relu(self.downsample5(max_pool_4))
        print("Conv 5 output shape : {}".format(conv_out_5.shape)) # [5, 1024, 14, 14]
        print("*****End of down pass*****")
        
        d_conv_out_1 = F.relu(self.deconv_w1(conv_out_5, output_size=conv_out_4.size()))
        print("deconv output 1 shape : {}".format(d_conv_out_1.shape))
        
        concat1 = torch.cat([conv_out_4, d_conv_out_1], dim=1)
        print("cancatenate output 1 shape : {}".format(concat1.shape))
        
        rout1 = F.relu(self.rconv_w1(concat1)) 
        print("rout1 shape {}".format(rout1.shape))
        
        d_conv_out_2 = F.relu(self.deconv_w2(rout1,output_size=conv_out_3.size() ))
        print("deconv output 2 shape : {}".format(d_conv_out_2.shape))
        
        concat2 = torch.cat([conv_out_3, d_conv_out_2], dim=1)
        print("cancatenate output 2 shape : {}".format(concat2.shape))
        
        rout2 = F.relu(self.rconv_w2(concat2)) 
        print("rout2 shape {}".format(rout2.shape))
        
        d_conv_out_3 = F.relu(self.deconv_w3(rout2,output_size=conv_out_2.size() ))
        print("deconv output 3 shape : {}".format(d_conv_out_3.shape))
        
        concat3 = torch.cat([conv_out_2, d_conv_out_3], dim=1)
        print("cancatenate output 3 shape : {}".format(concat3.shape))
        
        rout3 = F.relu(self.rconv_w3(concat3)) 
        print("rout3 shape {}".format(rout3.shape))

        d_conv_out_4 = F.relu(self.deconv_w4(rout3,output_size=conv_out_1.size() ))
        print("deconv output 4 shape : {}".format(d_conv_out_4.shape))
        
        concat4 = torch.cat([conv_out_1, d_conv_out_4], dim=1)
        print("cancatenate output 4 shape : {}".format(concat4.shape))
        
        rout4 = F.relu(self.rconv_w4(concat4)) 
        print("rout4 shape {}".format(rout4.shape))        
        
        out = F.relu(self.conv_last(rout4))
        print("output shape : {}".format(out.shape))        

#         out  is of shape (N, 6, 224, 224 ) How do i take one class from it as it's pixel wise distribution


#         scores = torch.zeros((5, 6)) #dummy output 

        return out


In [5]:
x = torch.zeros((5, 3, 224, 224), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]
model = ResUNet_trial(in_channel=3,num_classes=6)
scores = model(x)
print(scores.size())  # you should see [5,6,224,224]
scores_softmax = F.softmax(scores, 1)
print(scores_softmax.size())

Input shape : torch.Size([5, 3, 224, 224]), Conv. 1 output shape : torch.Size([5, 64, 224, 224]) 
Max. pool 1 output shape : torch.Size([5, 64, 112, 112])
Conv. 2 output shape : torch.Size([5, 128, 112, 112])
Max pool 2 output shape : torch.Size([5, 128, 56, 56])
Conv. 3 output shape : torch.Size([5, 256, 56, 56])
Max pool 3 output shape : torch.Size([5, 256, 28, 28])
Conv. 4 output shape : torch.Size([5, 512, 28, 28])
Max pool 4 output shape : torch.Size([5, 512, 14, 14])
Conv 5 output shape : torch.Size([5, 1024, 14, 14])
*****End of down pass*****
deconv output 1 shape : torch.Size([5, 512, 28, 28])
cancatenate output 1 shape : torch.Size([5, 1024, 28, 28])
rout1 shape torch.Size([5, 512, 28, 28])
deconv output 2 shape : torch.Size([5, 256, 56, 56])
cancatenate output 2 shape : torch.Size([5, 512, 56, 56])
rout2 shape torch.Size([5, 256, 56, 56])
deconv output 3 shape : torch.Size([5, 128, 112, 112])
cancatenate output 3 shape : torch.Size([5, 256, 112, 112])
rout3 shape torch.Size(

In [ ]:
def test_ResUNet():
    x = torch.zeros((5, 3, 224, 224), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]
    model = ResUNet_trial(in_channel=3,num_classes=6)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
    
test_ResUNet()

In [ ]:
# scores_softmax = F.softmax(scores, 1)
# print(scores_softmax.size())

# scores_softmax2 = F.softmax(scores.reshape(scores.size(0), scores.size(1), -1), 2).view_as(scores) 
# print(scores_softmax2.size())

# target = F.softmax(scores, dim=1) > 0.5
# labels = torch.argmax(target, dim=1)
target = torch.argmax(scores, dim=1)

print(target.shape)

In [ ]:
max_pool4_out = torch.randn(5, 1024, 14, 14)

# downsample = nn.Conv2d(512, 1024, 3, stride=1, padding=1)
# h = downsample(max_pool4_out)
# print(h.size())

upsample = nn.ConvTranspose2d(1024, 512, 3, stride=1, padding=1)

output = upsample(max_pool4_out, output_size=max_pool4_out.size())

print(output.size())

In [8]:
from collections import OrderedDict

import torch
import torch.nn as nn


class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

In [11]:
def test_UNet():
    x = torch.zeros((5, 3, 224, 224), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]
    model = UNet(in_channels=3)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
    
test_UNet()

torch.Size([5, 1, 224, 224])
